In [55]:
from openai import OpenAI

from agents.adjudicator import query as consult_players_handbook

In [30]:
tools = [
  {
      "type": "function",
      "function": {
          "name": "consult_players_handbook",
          "description": "Ask a question pertaining to the rules, setting, lore, or current situtation. Call this whenever you need more information.",
          "parameters": {
              "type": "object",
              "properties": {
                  "question": {
                      "type": "string",
                      "description": "A question that should be answered by consulting the available reference materials. Share all relevant details.",
                  },
              },
              "required": ["question"],
              "additionalProperties": False,
          },
      }
  }
]

In [83]:
messages = [
  {
      "role": "developer",
      "content": """
      You are a Dungeon Master overseeing a combat encounter. The player will tell you their action,
      and you will narrate the outcome.

      Use only the information you have been given to narrate the outcome. You may use the consult_players_handbook tool
      to get up-to-date information about the rules of the game, the setting, the mechanics, NPC stat sheets, and so on.

      Players:
      - Cedric van Zorndt (the Player)
        Human Rogue Lvl 1
        Armor Class 14
        Hit Points 9
        STR 12 (+1) - DEX 16 (+3) - CON 13 (+1) - INT 9 (-1) - WIS 12 (+1) - CHA 15 (+2)
        Proficiency bonus: +2
        Proficiencies:
        - Armor: light armor
        - Weapons: Simple weapons, hand crossbows, longswords, rapiers, shortswords
        - Tools: Thieves' tools
        Saving Throws: Dexterity, Intelligence
        Skills: Acrobatics, Deception, Sleight of Hand, Stealth
        Equipment: Shortsword, shortbow with a quiver of 20 arrows, burglar's pack, leather armor, two daggers, thieves' tools
        Expertise: Stealth, Thieves' tools
        Sneak attack 1d6
      """       
  },
  {
      "role": "user",
      "content": "I attack the Imp with my shortsword"
  }
]

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  tools=tools,
)

In [84]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_yJaiok4UFZgmZ8KXFKqsWw79', function=Function(arguments='{"question":"What are the armor class and hit points of an Imp in D&D 5e?"}', name='consult_players_handbook'), type='function')]

In [85]:
response.choices[0].message.content

In [87]:
import json
facts = consult_players_handbook(json.loads(response.choices[0].message.tool_calls[0].function.arguments)["question"])

/home/vscode/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.153'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [89]:
next_response = client.chat.completions.create(
  model="gpt-4o",
  messages=messages + [
      response.choices[0].message,
      {
        "role": "tool",
        "content": facts,
        "tool_call_id": response.choices[0].message.tool_calls[0].id
      }
  ],
  tools=tools,
)
next_response.choices[0].message.content

'Cedric, you draw your shortsword and move swiftly towards the Imp, exploiting your agility and training. You aim carefully at the mischievous creature, which has an Armor Class of 13.\n\nNow, roll a d20 and add your attack bonus (which is your proficiency bonus of +2 plus your Dexterity modifier of +3, totaling +5) to see if you hit the Imp.'

In [90]:
to_hit = client.chat.completions.create(
  model="gpt-4o",
  messages=messages + [
      response.choices[0].message,
      {
        "role": "tool",
        "content": facts,
        "tool_call_id": response.choices[0].message.tool_calls[0].id
      },
      next_response.choices[0].message,
      {
        "role": "user",
        "content": "I rolled a 9."
      }
  ],
  tools=tools,
)
to_hit.choices[0].message.content

"With your roll of 9, adding your attack bonus of +5, your total attack roll is 14. This surpasses the Imp's Armor Class of 13.\n\nYour shortsword strikes true, cutting into the Imp. Now roll for damage with your shortsword. Since your rogue class allows for Sneak Attack under certain conditions, please also note if any allies were in position adjacent to your target, or if the Imp was otherwise distracted. Otherwise, simply roll 1d6 for the shortsword's damage, adding your Dexterity modifier of +3."

In [92]:
damage_resp = client.chat.completions.create(
  model="gpt-4o",
  messages=messages + [
      response.choices[0].message,
      {
        "role": "tool",
        "content": facts,
        "tool_call_id": response.choices[0].message.tool_calls[0].id
      },
      next_response.choices[0].message,
      {
        "role": "user",
        "content": "I rolled a 9."
      },
      to_hit.choices[0].message,
      {
        "role": "user",
        "content": "I rolled a 6"
      }
  ],
  tools=tools,
)
damage_resp.choices[0].message.content

'With your roll of 6 and adding your Dexterity modifier of +3, the total damage dealt to the Imp is 9.\n\nYour strike is precise and powerful, as your shortsword slices into the Imp, leaving a deeply wounded gash. The Imp quivers in pain, its dark eyes narrowing in fury now that its Hit Points are diminished to 1.\n\nThe Imp, heavily injured but not defeated, flaps its leathery wings and locks its gaze on you, the desire for vengeance evident. Prepare for its counterattack!'

In [ ]:
"""Armor Class 13
        Hit Points 10
        STR 6 (-2) - DEX 17 (+3) - CON 13 (+1) - INT 11 (+0) - WIS 12 (+1) - CHA 14 (+2)
        Speed 20 (Fly 40) (20 in Rat form, 20 fly 60 in Raven form, 20 climb 20 in Spider form)
        Skills: Deception +4 Insight +3 Persusasion +4 Stealth +5
        Damage Resistances: Cold; bludgeoning, piercing and slashing from non-magical attacks that aren't silvered
        Damage Immunities: Fire, poison
        Condition Immunities: poisoned
        Senses: darkvision 120ft; Passive perception 11
        Languages: Infernal, Common
        Challenge: 1 (200 XP)
        Shapechanger: Te imp can use its action to polymorph into the beast form of a rat, a raven, or a spider, or into its devil form. Its statistics are
        the same for each form, except for the speed changes noted. Any equipment it is wearing or carrying is not transformed. It reverts to its devil form if it dies.
        Devil's Sight: Magical darkness doesn't impede the imp's darkvision.
        Magic Resistance: The imp has advantage on saving tghrows against spells and other magical effects.
        Actions:
        - Sting (Bite in Beast Form). Melee weapon Attack: +5 to hit, reach 5 ft, one creature. Hit: 5 (1d4+3) piercing damage, and the target must make a DC 11 Constitution saving throw, taking 10 (3d6) poison damage on a failed save, or half as much on a successful one.
        - Invisibility. The imp magically turns invisible until it attacks or until its concentration ends (as if concentrating on a spell). Any equipment the imp wears or carries is invisible with it."""